In [72]:
import re
from glob import glob

In [73]:
def strip_format(txt):
    texts = re.findall(r'[A-Z]*',txt)
    return ' '.join(' '.join(texts).split())

def cleanup(txt):
    if len(txt) < 2:
        return ""
    if "LOADING" in txt:
        return ""
    if "listen to this genre right now!" in txt:
        return ""
    #if "--- RECORDSEPARATOR ---" in txt:
    #    return ""
    return txt

def cleanup_track(txt):
    if "artist:" in txt:
        return ""
    if "--- RECORDSEPARATOR ---" in txt:
        return ""
    if "track:" in txt:
        return ""
    return txt

#def extract_track(txt):
    

In [ ]:
# For Tracks:

files = sorted([i.replace('../raw_data/','').replace('.swf','') for i in glob('../raw_data/*.swf')])    

for file in files:
    
    playlist = []
    
    track = {}
    
    num_guess = 0
    
    for fname in glob('../swf_data/'+file+'/*.txt'): # 902 multiline
        with open(fname) as f:
            txt = f.read()

            t = cleanup(txt)

            #if t is not "":
                #print (t)
                #print ("")
            
            #if str(txt).isdigit():
            #    num_guess = int(txt)
            
            if "artist:" in t:
                t = t.replace("\n--- RECORDSEPARATOR ---\n",'\t').split('\t')
                #print(t)
                track = {'artist':t[0].replace('artist:','').title(),
                         'title':t[1].replace('track:','').title(),
                         #'num': num_guess
                        }
                playlist.append(track)
                #print(track)
            
    
    for i, track in enumerate(playlist):
        print("{}\t{}\t{}\t{}".format(file, i+1, track['artist'], track['title']))

In [ ]:
# For Titles, and AKAs

files = sorted([i.replace('../raw_data/','').replace('.swf','') for i in glob('../raw_data/*.swf')])    

for file in files:

    parts = []
    
    for fname in glob('../swf_data/'+file+'/*.txt'): # 902 multiline
        with open(fname) as f:
            txt = f.read()

            t = cleanup_track(cleanup(txt)).title()
            if t is not "":
                parts.append(t)
    
    if len(parts) < 2:
        parts.append("")
        
    if len(parts) < 2:
        parts.append("")
        
    print(file, "\t", parts[0], "\t", parts[1].replace('Aka',''))

In [ ]:
[print(j) for j in sorted([i.replace('../raw_data/','').replace('.swf','') for i in glob('../raw_data/*.swf')])]

In [75]:
import pandas as pd
import numpy as np

In [ ]:
#for guide text descriptions:

df = pd.read_csv('../genres.csv')

for idx, r in df.iterrows():
    #print(r['type'], r['file'])
    
    with open('../raw_data/'+r['file']+'.txt') as f:
        desc = f.read()
        desc = desc[5:len(desc)-2] # get rid of text=, --
        #print(desc)
    
    if (r['node'] == r['title']):
        header = '##'
        if r['node'] in set(['House','Trance','Techno','Breakbeat','Jungle','Hardcore','Downtempo']):
            header = '#'
        print (header, r['node'])
    else:
        print ("##", r['node']+":", r['title'])

    
    type_of = ""    
    if str(r['scene']) != 'nan':
        type_of = "in the " + r['scene'] + " group,"
    
    if str(r['aka']) != 'nan':
        print("*aka: ", r['aka'].strip()+"*", type_of, 'emerged in', r['decade'])
    else:
        print(type_of)
    print()
    print(desc)
    print()

In [103]:
# Add descriptions to CSV
df = pd.read_csv('../genres.csv')

for idx, r in df.iterrows():
    #print(r['type'], r['file'])
    
    with open('../raw_data/'+r['file']+'.txt') as f:
        desc = f.read()
        desc = desc[5:len(desc)-2] # get rid of text=, --
        desc = desc.strip()

        df.at[idx,'description'] = desc
        
        #print(desc)
        
        #print('\n' in desc)
        #if '\n' in desc:
        #    print(desc)

df.to_csv('../genres.csv', index=False)
#df

In [86]:
# https://stackoverflow.com/a/33869154/11090908
def pandas_df_to_markdown_table(df):
    from IPython.display import Markdown, display
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    #display(Markdown(df_formatted.to_csv(sep="|", index=False)))
    return df_formatted.to_csv(sep="|", index=True)

In [104]:
df = pd.read_csv('../genres.csv')
#df = pd.read_csv('../links.csv')
#df = pd.read_csv('../tracks.csv')

df2 = df.describe(include='all').fillna(" ")
print(pandas_df_to_markdown_table(df2))
df2

|node|title|aka|type|scene|decade|file|description
0|---|---|---|---|---|---|---|---
count|187|187|129|187|96|180|187|187
unique|156|185|125|7|15|6|187|185
top|Tribal|Jungle|Tribal|Trance|Hard Dance|90s|hardcore|Like rats scrambling from a sinking ship, Progressive Trance and Anthem Trance producers escaped the banality of the builds, breakdowns and chorus melodies when it was declared by the genre's elite (read: Sasha and Digweed) as being thoroughly uncool to keep playing the stuff. So they latched onto the other genres as a way to stay ahead of the pack. Break Trance (Breakbeat), Tech Trance (Techno) and Deep Trance (House) are such genres created in the wake of the Great Anthem crash.
freq|4|2|4|37|10|121|1|3



,node,title,aka,type,scene,decade,file,description
count,187,187,129,187,96,180,187,187
unique,156,185,125,7,15,6,187,185
top,Tribal,Jungle,Tribal,Trance,Hard Dance,90s,hardcore,"Like rats scrambling from a sinking ship, Prog..."
freq,4,2,4,37,10,121,1,3
